# Analysis of movie budget relative to primary outcomes: Movie Ratings, Gross Revenue

Importing packages and reading in data

In [1]:
#importing necessary packages
import pandas as pd
import matplotlib as plt
import sqlite3

In [2]:
#file paths for importing data
#NOTE that some unzipped files are too large for the github repo and must be extracted locally before analysis can be performed
movie_budget_file_path = '~/Documents/Flatiron/phase_1/Movie_Studio_DATA/tn.movie_budgets.csv.gz'
microsoft_movie_studio_db_file_path = './../Movie_Studio_DATA/im.db'
msms_db_conn = sqlite3.connect(microsoft_movie_studio_db_file_path)

In [3]:
#reading in movie budget csv for outcome data
movie_budget_df = pd.read_csv(movie_budget_file_path)
#check to see if read properly
movie_budget_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [4]:
#execute SQL query to obtain a dataframe with movie names and avg ratings, filtered for original titles
avg_ratings_query = """
SELECT movie_basics.primary_title, movie_ratings.*
FROM movie_basics
INNER JOIN movie_ratings ON movie_basics.movie_id = movie_ratings.movie_id 
INNER JOIN movie_akas ON movie_ratings.movie_id = movie_akas.movie_id
WHERE is_original_title != 0.0
"""

microsoft_movie_studio_avg_ratings = pd.read_sql(avg_ratings_query, msms_db_conn)

#check query results
microsoft_movie_studio_avg_ratings

,primary_title,movie_id,averagerating,numvotes
0,Jurassic World,tt0369610,7.0,539338
1,John Carter,tt0401729,6.6,241792
2,Versailles Rediscovered - The Sun King's Vanis...,tt10010134,7.0,5
3,Alice in Wonderland,tt1014759,6.5,358813
4,From Paris with Love,tt1179034,6.5,107079
...,...,...,...,...
36890,Der Atem,tt9644084,5.2,5
36891,Homing,tt9657946,6.0,16
36892,Anderswo. Allein in Afrika,tt9723084,6.7,35
36893,Monkey King: The Volcano,tt9726638,6.0,6


Below are some descriptive methods used to determine what is necessary to clean this dataset for final analysis

In [5]:
microsoft_movie_studio_avg_ratings.describe()

,averagerating,numvotes
count,36895.000000,3.689500e+04
mean,6.173877,6.956449e+03
std,1.337955,4.252235e+04
min,1.000000,5.000000e+00
25%,5.400000,3.700000e+01
50%,6.300000,1.850000e+02
75%,7.100000,1.032000e+03
max,10.000000,1.841066e+06


In [6]:
movie_budget_df.describe()

,id
count,5782.000000
mean,50.372363
std,28.821076
min,1.000000
25%,25.000000
50%,50.000000
75%,75.000000
max,100.000000


In [7]:
microsoft_movie_studio_avg_ratings["primary_title"].value_counts()

Home                     7
Paradise                 7
Together                 7
Gold                     6
Brothers                 6
                        ..
Habana muda              1
Proshchatsya ne budem    1
Marriage Blue            1
Meciar                   1
David's Secret           1
Name: primary_title, Length: 35419, dtype: int64

Begin Data Cleaning:

In [8]:
#attempt to merge dataframes along movie titles. Inner merge will produce a dataframe that only includes movies for which a complete dataset is available.
movie_rating_budget_df = pd.merge(microsoft_movie_studio_avg_ratings, movie_budget_df, how='inner', left_on='primary_title', right_on='movie' )

In [9]:
movie_rating_budget_df 
#note the presence of a non-null-type null value in row 3 for gross sales. 
#These, and similar data, must be identified and dropped from the dataset after conversion to a numeric value.

,primary_title,movie_id,averagerating,numvotes,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,Jurassic World,tt0369610,7.0,539338,34,"Jun 12, 2015",Jurassic World,"$215,000,000","$652,270,625","$1,648,854,864"
1,John Carter,tt0401729,6.6,241792,14,"Mar 9, 2012",John Carter,"$275,000,000","$73,058,679","$282,778,100"
2,Alice in Wonderland,tt1014759,6.5,358813,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
3,Alice in Wonderland,tt1014759,6.5,358813,60,"Jul 28, 1951",Alice in Wonderland,"$3,000,000",$0,$0
4,The Bourne Legacy,tt1194173,6.7,268678,76,"Aug 10, 2012",The Bourne Legacy,"$125,000,000","$113,203,870","$280,355,920"
...,...,...,...,...,...,...,...,...,...,...
2317,Thoroughbreds,tt5649108,6.7,25098,47,"Mar 9, 2018",Thoroughbreds,"$5,000,000","$3,072,605","$3,187,302"
2318,Aladdin,tt6139732,7.4,57549,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
2319,Aladdin,tt6139732,7.4,57549,33,"Nov 11, 1992",Aladdin,"$28,000,000","$217,350,219","$504,050,219"
2320,November,tt6164502,7.2,2455,81,"Jul 22, 2005",November,"$250,000","$191,862","$191,862"


In [10]:
#Identified a number of duplicate entries
movie_rating_budget_df["primary_title"].value_counts()

Home                 21
Brothers             12
The Gift              6
Crossroads            6
Robin Hood            6
                     ..
Freeway               1
Rodeo Girl            1
The Help              1
Act of Valor          1
The Expendables 3     1
Name: primary_title, Length: 1943, dtype: int64

In [11]:
#Removing duplicate entries
movie_rating_budget_df.drop_duplicates(subset="primary_title", inplace=True)

In [13]:
#evaluating clean dataset
movie_rating_budget_df

,primary_title,movie_id,averagerating,numvotes,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,Jurassic World,tt0369610,7.0,539338,34,"Jun 12, 2015",Jurassic World,"$215,000,000","$652,270,625","$1,648,854,864"
1,John Carter,tt0401729,6.6,241792,14,"Mar 9, 2012",John Carter,"$275,000,000","$73,058,679","$282,778,100"
2,Alice in Wonderland,tt1014759,6.5,358813,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
4,The Bourne Legacy,tt1194173,6.7,268678,76,"Aug 10, 2012",The Bourne Legacy,"$125,000,000","$113,203,870","$280,355,920"
5,Limitless,tt1219289,7.4,492490,91,"Mar 18, 2011",Limitless,"$27,000,000","$79,249,455","$155,564,919"
...,...,...,...,...,...,...,...,...,...,...
2316,The Big Sick,tt5462602,7.6,104008,64,"Jun 23, 2017",The Big Sick,"$5,000,000","$42,872,467","$53,972,860"
2317,Thoroughbreds,tt5649108,6.7,25098,47,"Mar 9, 2018",Thoroughbreds,"$5,000,000","$3,072,605","$3,187,302"
2318,Aladdin,tt6139732,7.4,57549,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
2320,November,tt6164502,7.2,2455,81,"Jul 22, 2005",November,"$250,000","$191,862","$191,862"


In [14]:
movie_rating_budget_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1943 entries, 0 to 2321
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   primary_title      1943 non-null   object 
 1   movie_id           1943 non-null   object 
 2   averagerating      1943 non-null   float64
 3   numvotes           1943 non-null   int64  
 4   id                 1943 non-null   int64  
 5   release_date       1943 non-null   object 
 6   movie              1943 non-null   object 
 7   production_budget  1943 non-null   object 
 8   domestic_gross     1943 non-null   object 
 9   worldwide_gross    1943 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 167.0+ KB


In [15]:
movie_rating_budget_df.describe()

,averagerating,numvotes,id
count,1943.000000,1.943000e+03,1943.000000
mean,6.232578,9.078380e+04,50.695317
std,1.091510,1.507041e+05,28.559094
min,1.600000,5.000000e+00,1.000000
25%,5.600000,3.112000e+03,26.000000
50%,6.300000,3.475900e+04,51.000000
75%,7.000000,1.084740e+05,75.000000
max,9.000000,1.841066e+06,100.000000


For the final analysis, gross revenues and budget values need to be converted to numeric data types. This will allow us to calculate the net profits for each movie in our dataset, as well as address some non-null-type null values identified earlier in the cleaning process. 

To control for the wide time range that this dataset covers, net profits will be evaluated as a percentage of the movie budget, and identified as "return on investment".